# Access to Ambre synthetic spectra for photometric corrections

- Sylvie Dagoret-Campagne
- creation date : 2024-12-08
- last update : 2024-12-10 (find the contact)

In [ ]:
# Access astronomical databases
from pyvo import registry  # version >=1.4.1 

# Moc and HEALPix tools
from mocpy import MOC

# Sky visualization
from ipyaladin import Aladin    # version >=0.3.0

# For plots
import matplotlib.pyplot as plt

In [ ]:
# Astronomy tools
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import vstack

# Access astronomical databases
#import pyvo
#from astroquery.simbad import Simbad
#from astroquery.vizier import Vizier
#from astroquery.xmatch import XMatch

# Sky visualization
#from ipyaladin import Aladin


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

--------------------------------------------------------

Modify the <a href='https://vizier.cds.unistra.fr/'>VizieR catalogue</a> name (variable <b>CATALOGUE</b>) and anything else you might feel like exploring!

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1

**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [ ]:
# the catalogue name in VizieR
CATALOGUE = "J/A+A/542/A48"

We first retrieve catalogue information.

In [ ]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [ ]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [ ]:
voresource.creators[0]

In [ ]:
voresource.creators[1]

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [ ]:
tables = voresource.get_tables()
tables

In [ ]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

The actual data can then be access using any of the ``access_modes`` of the voresource.

In [ ]:
voresource.access_modes()

The web access is found by following the ``reference_url``

In [ ]:
voresource.reference_url

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [ ]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = voresource.get_service("tap").run_sync(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

In [ ]:
tap_records.to_table()

<!-- section position -->

We will explore the other access modes here.

### 3.2. Execute a cone search query

We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) protocol of the virtual observatory (only available for tables with positions).

Adapt the radius and position of the center to your needs.

In [ ]:
conesearch_radius = 1 / 60.0  # in degrees
conesearch_center = (45.0, 7.726245081475826)

In [ ]:
conesearch_records = voresource.get_service("conesearch").search(
    pos=conesearch_center,
    sr=conesearch_radius,
)
conesearch_records

In [ ]:
conesearch_records.to_table()

## 4. Get the catalogue coverage

VizieR also provides the coverage of each catalogue with Multiple Order Coverage (<a href='https://ivoa.net/documents/MOC/'>MOC</a>) at order 10 -- meaning with a 0.001 rad spacial resolution.

In [ ]:
# retrieve the MOC
catalogue_coverage = MOC.from_fits(
    f"https://cdsarc.cds.unistra.fr/viz-bin/moc/{CATALOGUE}",
)
catalogue_coverage.display_preview()

We can plot the coverage with ``matplotlib``.

In [ ]:
fig = plt.figure(figsize=(5, 5))
wcs = catalogue_coverage.wcs(fig)
ax = fig.add_subplot(projection=wcs)
catalogue_coverage.fill(ax=ax, wcs=wcs, alpha=0.5, color="blue")

or with ipyaladin

In [ ]:
aladin = Aladin()
aladin

We can add the tables and coverage in this ipyaladin window.

In [ ]:
aladin.add_table(conesearch_records.to_table())

In [ ]:
aladin.add_table(tap_records.to_table())

In [ ]:
aladin.add_moc_from_URL(
    f"https://cdsarc.cds.unistra.fr/viz-bin/moc/{CATALOGUE}",
    moc_options={"opacity": 0.5, "color": "pink"},
)

For more information about manipulating coverages, see [MOCpy documentation](https://cds-astro.github.io/mocpy/).

## 5.) Try to access to Spectra associated to this catalog

In [ ]:
catalogue_ivoid 

### Here I change the reference to the catalog to be that given in the documentation

https://pollux.oreme.org/static/polluxwebapp/files/User's%20guide_v12.pdf

The POLLUX database can also be accessed via
the VizieR service at Centre de Donnees de Strasbourg (CDS). 

- It is a registered service of the VO :
- ivo://ov-gso/ssap/pollux

In [ ]:
catalogue_ivoid = f"ivo://ov-gso/ssap/pollux"
# the actual query to the registry
registry_pollux = registry.search(ivoid=catalogue_ivoid)

In [ ]:
registry_pollux = registry.search(ivoid=catalogue_ivoid)

In [ ]:
registry_pollux.get_summary()

In [ ]:
voresource = registry_pollux[0]

In [ ]:
voresource

In [ ]:
voresource.get_contact()

In [ ]:
voresource.describe(verbose=True)

In [ ]:
#web_service = voresource.get_service("web")
ssap_service = voresource.get_service("ssa")

Example of SSA query search

- http://www.myvo.org/ssa?REQUEST=queryData&POS=22.438,-17.2&SIZE=0.02

- https://pollux.oreme.org/vo/ssa?REQUEST=queryData&POS=22.438,-17.2&SIZE=0.02

- https://pollux.oreme.org/vo/ssa?REQUEST=queryData

In [ ]:
ssap_service.search?

In [ ]:
#mast_ssa_service = pyvo.dal.SSAService(
#    "https://archive.stsci.edu/ssap/search2.php?id=HST&",
#)
#diameter = u.Quantity(2 * 40.0, unit="arcmin")
#position = SkyCoord.from_name("A1656")
#mast_hst_results = mast_ssa_service.search(pos=position, diameter=diameter)
#mast_hst_results

==> Now I am sure I have an object here in 

In [ ]:
conesearch_records.to_table()

In [ ]:
filename = conesearch_records["OrigFile"][0]
filename

In [ ]:
position = SkyCoord(conesearch_center[0], conesearch_center[1], unit="deg")
diameter = u.Quantity(2 * 5.0, unit="arcmin")

In [ ]:
ssap_query = ssap_service.create_query(pos=position,diameter=diameter)

In [ ]:
ssap_result = ssap_service.search(pos=position,diameter=diameter)

In [ ]:
ssap_result = ssap_service.search(id="FEROS.2009-08-27T09:19:37.279")

In [ ]:
ssap_result = ssap_service.search(id="083.A-9011(B)")

In [ ]:
ssap_result = ssap_service.search(id=filename)

In [ ]:
ssap_result = ssap_service.search(id="HD18632")	